In [12]:
import pandas as pd
import requests

## Interaction Novelty

#### Load Data

In [55]:
df = pd.read_excel('data/final_results_test3.xlsx').drop(labels='Unnamed: 0',axis=1)
df.head(5)

,pmid,drug_name,gene_name,interaction_occurs_with_gene,interaction_type,evidence,gene_concept,gene_label,gene_match_type,drug_concept,drug_label,drug_match_type
0,37726279,venetoclax,ABCC1,YES,INHIBITING,Genetic and pharmacologic ABCC1 inactivation p...,normalize.gene.hgnc:51,ABCC1,100,normalize.therapy.rxcui:1747556,venetoclax,80
1,37726279,glutathione,ABCC1,YES,ACTIVATING,Consistent with ABCC1-specific export of gluta...,normalize.gene.hgnc:51,ABCC1,100,normalize.therapy.rxcui:4890,glutathione,80
2,37004989,Kynurenine,AhR,YES,ACTIVATING,"An endogenous AhR ligand, kynurenine (Kyn), wa...",normalize.gene.hgnc:348,AHR,100,normalize.therapy.drugbank:DB02070,Kynurenine,80
3,33932119,ONC201,AKT,YES,INHIBITING,"The compensatory, pro-survival PI3K/AKT/mTOR p...",normalize.gene.hgnc:391,AKT1,60,normalize.therapy.iuphar.ligand:9978,ONC201,80
4,26884600,ONC201,AKT,YES,INHIBITING,ONC201 (also called TIC10) is a small molecule...,normalize.gene.hgnc:391,AKT1,60,normalize.therapy.iuphar.ligand:9978,ONC201,80


In [11]:
df[['drug_name','gene_name','pmid']]

,drug_name,gene_name,pmid
0,venetoclax,ABCC1,37726279
1,glutathione,ABCC1,37726279
2,Kynurenine,AhR,37004989
3,ONC201,AKT,33932119
4,ONC201,AKT,26884600
...,...,...,...
132,ONC201,TRAIL,26884600
133,melphalan,Trip13,37942576
134,panobinostat,Trip13,37942576
135,galangin,TRPV1,39047882


#### GraphQL

In [19]:

GQL_URL = "https://dgidb.org/api/graphql"

GQL_QUERY = """
query GeneInteractions($genes: [String!]!) {
  genes(names: $genes) {
    nodes {
      name
      conceptId
      interactions {
        drug {
          name
          conceptId
          approved
        }
        gene {
          name
          conceptId
          longName
        }
        interactionScore
        interactionTypes { type directionality }
        interactionAttributes { name value }
        publications { pmid }
        sources { sourceDbName }
      }
    }
  }
}
"""

def fetch_gene_interactions(genes):
    payload = {"query": GQL_QUERY, "variables": {"genes": genes}}
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    r = requests.post(GQL_URL, json=payload, headers=headers)
    r.raise_for_status()
    return r.json()


In [17]:
genes = list(df['gene_label'].unique())
genes[0:5]

['ABCC1', 'AHR', 'AKT1', 'AKT2', 'PRKAA2']

In [ ]:
response = fetch_gene_interactions(genes)

In [22]:
genes_data = response.get("data", {}).get("genes", {}).get("nodes", [])

rows = []
for gene_node in genes_data:
    gene_name = gene_node.get("name")
    gene_id = gene_node.get("conceptId")
    for interaction in gene_node.get("interactions", []):
        drug = interaction.get("drug") or {}
        igene = interaction.get("gene") or {}
        interaction_types = interaction.get("interactionTypes") or []
        interaction_attributes = interaction.get("interactionAttributes") or []
        publications = interaction.get("publications") or []
        sources = interaction.get("sources") or []

        rows.append({
            "gene_name": gene_name,
            "gene_concept_id": gene_id,
            "interaction_gene_name": igene.get("name"),
            "interaction_gene_concept_id": igene.get("conceptId"),
            "interaction_gene_long_name": igene.get("longName"),
            "drug_name": drug.get("name"),
            "drug_concept_id": drug.get("conceptId"),
            "drug_approved": drug.get("approved"),
            "interaction_score": interaction.get("interactionScore"),
            "interaction_types": ";".join(
                f"{t.get('type')}({t.get('directionality')})"
                for t in interaction_types
                if t
            ),
            "interaction_attributes": ";".join(
                f"{a.get('name')}={a.get('value')}"
                for a in interaction_attributes
                if a
            ),
            "pmids": ";".join(str(p.get("pmid")) for p in publications if p),
            "sources": ";".join(s.get("sourceDbName") for s in sources if s),
        })

data = pd.DataFrame(rows)
data.head()

,gene_name,gene_concept_id,interaction_gene_name,interaction_gene_concept_id,interaction_gene_long_name,drug_name,drug_concept_id,drug_approved,interaction_score,interaction_types,interaction_attributes,pmids,sources
0,MYD88,hgnc:7562,MYD88,hgnc:7562,MYD88 innate immune signal transduction adaptor,ZANUBRUTINIB,rxcui:2262435,True,4.350317,,,,PharmGKB
1,MYD88,hgnc:7562,MYD88,hgnc:7562,MYD88 innate immune signal transduction adaptor,FENTANYL CITRATE,rxcui:142436,True,1.160084,,,26332828,PharmGKB
2,MYD88,hgnc:7562,MYD88,hgnc:7562,MYD88 innate immune signal transduction adaptor,IBRUTINIB,rxcui:1442981,True,1.535406,,Alteration=MYD88:L265P,,CGI;CIViC;PharmGKB
3,MAP2K7,hgnc:6847,MAP2K7,hgnc:6847,mitogen-activated protein kinase kinase 7,DABRAFENIB,rxcui:1424911,True,0.193347,inhibitor(INHIBITORY),Clinical Trial ID=NCT01701037;Cancer Type=Mela...,,MyCancerGenomeClinicalTrial
4,MAP2K7,hgnc:6847,MAP2K7,hgnc:6847,mitogen-activated protein kinase kinase 7,SELUMETINIB,rxcui:2289380,True,0.158193,inhibitor(INHIBITORY),Clinical Trial ID=NCT01586624;Cancer Type=Lung...,,MyCancerGenomeClinicalTrial


#### Assess Pairs

In [23]:
ai_interactions = df[['drug_name','gene_name','pmid']]
ai_interactions.head(5)

,drug_name,gene_name,pmid
0,venetoclax,ABCC1,37726279
1,glutathione,ABCC1,37726279
2,Kynurenine,AhR,37004989
3,ONC201,AKT,33932119
4,ONC201,AKT,26884600


In [30]:
ai_interactions['drug_name'] = ai_interactions['drug_name'].apply(lambda x: x.lower())
ai_interactions['gene_name'] = ai_interactions['gene_name'].apply(lambda x: x.lower())
data['drug_name'] = data['drug_name'].apply(lambda x: x.lower())
data['gene_name'] = data['gene_name'].apply(lambda x: x.lower())

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_54608/1656402764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ai_interactions['drug_name'] = ai_interactions['drug_name'].apply(lambda x: x.lower())
/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_54608/1656402764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ai_interactions['gene_name'] = ai_interactions['gene_name'].apply(lambda x: x.lower())


In [37]:
ai_interactions['novel?'] = 'Not Evaluated'
for idx, row in ai_interactions.iterrows():
    tdf = data[data['gene_name']==row['gene_name']].reset_index()
    tdf = tdf[tdf['drug_name']==row['drug_name']].reset_index()
    if len(tdf) > 0:
        ai_interactions.at[idx, 'novel?'] = False
    if len(tdf) == 0:
        ai_interactions.at[idx, 'novel?'] = True

ai_interactions['novel?'].value_counts()

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_54608/1146508370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ai_interactions['novel?'] = 'Not Evaluated'


novel?
True     121
False     16
Name: count, dtype: int64

In [38]:
ai_interactions

,drug_name,gene_name,pmid,novel?
0,venetoclax,abcc1,37726279,True
1,glutathione,abcc1,37726279,True
2,kynurenine,ahr,37004989,True
3,onc201,akt,33932119,True
4,onc201,akt,26884600,True
...,...,...,...,...
132,onc201,trail,26884600,True
133,melphalan,trip13,37942576,True
134,panobinostat,trip13,37942576,True
135,galangin,trpv1,39047882,True


#### Graph

In [48]:
import plotly.graph_objects as go

ai_interactions["novel?"] = ai_interactions["novel?"].astype(bool)

novel_true = ai_interactions["novel?"].sum()
novel_false = (~ai_interactions["novel?"]).sum()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=[novel_false],
    y=["Interactions"],
    name="Already Known",
    orientation="h",
    marker=dict(color="firebrick"),
    text=[novel_false],
    textposition="inside",
    insidetextanchor="middle",
    textfont=dict(color="white", size=14)
))

fig.add_trace(go.Bar(
    x=[novel_true],
    y=["Interactions"],
    name="Novel",
    orientation="h",
    marker=dict(color="royalblue"),
    text=[novel_true],
    textposition="inside",
    insidetextanchor="middle",
    textfont=dict(color="white", size=14)
))

fig.update_layout(
    barmode="stack",
    template="simple_white",
    title=dict(
        text="Novelty of AI Curated Interactions",
        x=0.5,
        font=dict(size=20)
    ),
    xaxis=dict(
        title="Number of Interactions",
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        mirror=False
    ),
    yaxis=dict(
        showgrid=False,
        linecolor="black",
        mirror=False
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    bargap=0.05,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=14)
    )
)

fig.update_traces(marker_line_width=0)

fig.write_image("graphs/novel_interactions-d1.png", scale=3, width=800, height=300)

fig.show()


/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_54608/3163831991.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## PMID

In [54]:
df

,id,pmid,citation,created_at,updated_at


In [56]:
import psycopg2
import pandas as pd
import numpy as np

# 1) Prepare PMIDs from your DataFrame
pmids = pd.Series(df['pmid']).dropna().astype(str).unique().tolist()

# 2) Connect
conn = psycopg2.connect(
    dbname="dgidb_2025",
    user="mjc014",
    password="",
    host="localhost",
    port="5432"
)

# 3) Query: cast column -> text so it matches the text[] param
query = """
SELECT *
FROM public.publications
WHERE pmid::text = ANY(%s)
ORDER BY id ASC;
"""

df = pd.read_sql(query, conn, params=(pmids,))
conn.close()

print(f"Found {len(df)} PMIDs already in DGIdb")
display(df.head())

# 4) Novelty %
found_pmids = set(df['pmid'].astype(str).unique())
all_pmids   = set(pmids)
novel_pmids = sorted(all_pmids - found_pmids)

novelty_pct = 100.0 * (len(novel_pmids) / max(1, len(all_pmids)))
print(f"Novel PMIDs: {len(novel_pmids)} / {len(all_pmids)} = {novelty_pct:.2f}%")


Found 0 PMIDs already in DGIdb


/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_54608/3631050725.py:25: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,id,pmid,citation,created_at,updated_at


Novel PMIDs: 83 / 83 = 100.00%
